In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd
from sklearn.linear_model import LinearRegression as LR
import gc
from scipy.stats import pearsonr as corr
from os.path import exists
from tqdm.contrib.concurrent import process_map
from multiprocessing import Pool
import os
import logging

In [2]:
returns = 'trt1m' #use log instead
eps = 1e-12

In [3]:
CRSP  = pd.read_csv('CRSP.csv', usecols=['pstkq', 'txditcq', 'fyearq','fqtr','conm', 'atq', 'cshoq', 'ibq', 'prchq', 'tic','rdq', 'teqq', 'datadate'], parse_dates=['rdq', 'datadate'])
Sec_m = pd.read_csv('Sec.csv' , usecols=['prccm', 'prchm', 'trt1m', 'trfm', 'cshtrm', 'tic','datadate'], parse_dates=['datadate'])
ff_m  = pd.read_csv('F-F.csv', skiprows=3)[:-98]

In [4]:
#CRSP                = CRSP.sort_values('datadate') #drop index na instead
CRSP = CRSP.set_index(['tic','datadate'])
CRSP = CRSP.sort_index()
CRSP = CRSP.loc[CRSP.index.dropna()]
CRSP = CRSP.drop_duplicates()
CRSP                = CRSP.reset_index().sort_values('datadate')
for i in CRSP.columns:
    if i not in ['tic', 'datadate', 'rdq', 'conm']:
        CRSP[i]      = pd.to_numeric(CRSP[i])

Sec_m               = Sec_m.sort_values('datadate')
for i in Sec_m.columns:
    if i not in ['tic', 'datadate']:
        Sec_m[i]      = pd.to_numeric(Sec_m[i])

ff_m                = ff_m.dropna()
ff_m['date']        = pd.to_datetime(ff_m[ff_m.columns[0]], format='%Y%m')
ff_m                = ff_m.drop(columns=ff_m.columns[0], axis=1)
for i in ff_m.columns:
    if i not in ['date']:
        ff_m[i]      = pd.to_numeric(ff_m[i])

In [5]:
df = pd.merge_asof(Sec_m, CRSP, on='datadate', by='tic')
df = pd.merge_asof(df, ff_m, right_on='date', left_on='datadate')

In [6]:
df = df.sort_values(['tic', 'datadate']).set_index(['tic','datadate'])

In [7]:
ff_d                = pd.read_csv('F-F_d.csv', skiprows=3)[:-1]
ff_d                = ff_d.dropna()
ff_d['date']        = pd.to_datetime(ff_d[ff_d.columns[0]], format='%Y%m%d')
ff_d['Mkt-RF']      = pd.to_numeric(ff_d['Mkt-RF'])
ff_d['SMB']         = pd.to_numeric(ff_d['SMB'])
ff_d['HML']         = pd.to_numeric(ff_d['HML'])
ff_d['RF']          = pd.to_numeric(ff_d['RF'])

#df_d = pd.merge_asof(Sec_d, ff_d, right_on='date', left_on='datadate')
#df_d = df_d.dropna().sort_values(['tic', 'datadate']).set_index(['tic','datadate'])

In [ ]:
def single_sub(sub): # .loc[['AAPL','GME']]
    try:
        sub = sub.copy()
        sub = sub.reset_index().set_index('datadate')
        tic = np.unique(sub['tic'])[0]

        file = f'data/{tic}_daily.csv'
        e = exists(file)

        if e:
            Sec_d = pd.read_csv(file, usecols=['prccd', 'cshtrd', 'trfd', 'tic','datadate'], parse_dates=['datadate'])
            Sec_d = Sec_d.set_index(['datadate'])
            Sec_d = Sec_d.sort_index()
            Sec_d = Sec_d.loc[Sec_d.index.dropna()]
            Sec_d = Sec_d.drop_duplicates()
            Sec_d = Sec_d.reset_index()
            df_d = pd.merge_asof(Sec_d[Sec_d['tic'] == tic], ff_d, right_on='date', left_on='datadate')
            df_d = df_d.set_index(['datadate'])
            df_d['returns'] = df_d['prccd'] * df_d['trfd']
            df_d['returns'] = df_d['returns']/(df_d['returns'].shift(1)+eps)
            df_d['returns'] = (df_d['returns'] - 1) * 100

        sub['r2_1'] = sub[returns].shift(2)
        sub['r36_13'] = 0.0
        for i in range(13, 37):
            sub['r36_13'] += sub[returns].shift(i)
        sub['r12_7'] = 0.0
        for i in range(7, 13):
            sub['r12_7'] += sub[returns].shift(i)
        sub['r12_2'] = sub['r12_7']
        for i in range(2, 7):
            sub['r12_2'] += sub[returns].shift(i)
        sub['ST_Rev'] = sub[returns].shift(1)
        sub['LT_Rev'] = sub['r36_13']
        for i in range(37, 61):
            sub['LT_Rev'] += sub[returns].shift(i)
        sub['AT'] = sub['atq']
        sub['LME'] = sub['prccm'] * sub['cshoq']
        sub['LTurnover'] = sub['cshtrm'] / (sub['cshoq']+eps)

        sub['Beta'] = np.nan
        sub['Resid_Var'] = np.nan
        sub['IdioVol'] = np.nan
        sub['SUV'] = np.nan
        sub['Variance'] = np.nan
        if e:
            for i in range(61, len(sub)):
                #TODO: They use Log excess returns - which don't make sense - since excess returns can be negative
                daily = df_d.loc[sub.index[max(0, i-60)]:sub.index[i]].reset_index()[['Mkt-RF', 'returns', 'RF', 'date']].dropna()
                if len(daily) < 750:
                    continue
                excess_ret = daily['returns'] - daily['RF']
                #log_excess_ret = np.log(excess_ret)
                excess_ret_3 = excess_ret + excess_ret.shift(1) + excess_ret.shift(2)
                #log_excess_ret_3 = np.log(excess_ret_3)
                #mkt_excess_ret = daily['Mkt-RF']
                mkt_excess_ret = daily['Mkt-RF']
                mkt_excess_ret_3 = mkt_excess_ret + mkt_excess_ret.shift(1) + mkt_excess_ret.shift(2)
                #mkt_log_excess_ret_3 = np.log(mkt_excess_ret_3)
                try:
                    sub.loc[sub.index[i], 'Beta'] = corr(excess_ret_3.iloc[2:], mkt_excess_ret_3.iloc[2:])[0] * np.std(excess_ret.iloc[2:][daily.iloc[2:]['date'] >= sub.index[i-12]]) / (np.std(excess_ret.iloc[2:][daily.iloc[2:]['date'] >= sub.index[i-12]])+eps)
                except Exception as e:
                    print(daily, excess_ret_3.iloc[2:], mkt_excess_ret_3.iloc[2:])
                    logging.exception(e)
                    raise

            for i in range(12, len(sub)):
                daily = df_d.loc[sub.index[max(0, i-12)]:sub.index[i]].reset_index()[['Mkt-RF', 'SMB', 'HML', 'returns', 'RF']].dropna()
                if len(daily) < 120:
                    continue
                reg =  LR().fit(daily[['Mkt-RF', 'SMB', 'HML']], (daily['returns'] - daily['RF']))
                pred = reg.predict(daily[['Mkt-RF', 'SMB', 'HML']])
                residual = (daily['returns'] - daily['RF'] - pred)
                sub.loc[sub.index[i], 'IdioVol'] = np.std(residual)


            for i in range(2, len(sub)):
                daily = df_d.loc[sub.index[max(0, i-2)]:sub.index[i]].reset_index()[['Mkt-RF', 'SMB', 'HML', 'returns', 'RF']].dropna()
                if len(daily) < 22:
                    continue
                reg =  LR().fit(daily[['Mkt-RF', 'SMB', 'HML']], (daily['returns'] - daily['RF']))
                pred = reg.predict(daily[['Mkt-RF', 'SMB', 'HML']])
                residual = (daily['returns'] - daily['RF'] - pred)
                sub.loc[sub.index[i], 'Resid_Var'] = np.std(residual)

            for i in range(13, len(sub)):
                daily = df_d.loc[sub.index[max(0, i-13)]:sub.index[i-1]].reset_index()[['cshtrd', 'returns', 'RF']].dropna()
                if len(daily) < 120:
                    continue
                daily2 = df_d.loc[sub.index[max(0, i-1)]:sub.index[i]].reset_index().dropna()
                if len(daily2) < 10:
                    continue
                reg =  LR().fit(np.abs(daily['returns'] - daily['RF']).values.reshape(-1, 1), daily['cshtrd'])
                pred = reg.predict((daily2['returns'] - daily2['RF']).values.reshape(-1, 1))
                residual = (daily2['cshtrd'] - pred)
                sub.loc[sub.index[i], 'SUV'] = np.mean(residual) / (np.std(residual)+eps)

            for i in range(2, len(sub)):
                daily = df_d.loc[sub.index[max(0, i-2)]:sub.index[i]].reset_index()
                if len(daily) < 22:
                    continue
                sub.loc[sub.index[i], 'Variance'] = np.var(daily['returns'])
            #TODO: Use monthly data where daily is unavailable

        sub['MktBeta'] = np.nan
        for i in range(61, len(sub)):
            slice_ = sub.iloc[i-61:i][['Mkt-RF', 'SMB', 'HML', 'RF', returns]].dropna()
            if len(slice_) < 30:
                    continue
            sub.loc[sub.index[i], 'MktBeta'] = LR().fit(slice_[['Mkt-RF', 'SMB', 'HML']], (slice_[returns] - slice_['RF'])).coef_[0]

        sub['Rel2High'] = sub['prccm']/(sub['prchm'].rolling(12).max()+eps)
        sub['ROE'] = sub['ibq'].rolling(4).sum()/((sub['teqq']+sub['txditcq']-sub['pstkq']).shift(4)+eps)

        sub = sub.reset_index()
        #df2.append(sub[['datadate', 'tic', 'fyearq', 'fqtr', 'conm', 'datadate', 'rdq', 'r2_1', 'r36_13', 'r12_7', 'r12_2', 'ST_Rev', 'LT_Rev', 'AT', 'LME', 'LTurnover', 'Beta', 'MktBeta', 'IdioVol', 'Resid_Var', 'SUV', 'Rel2High', 'ROE', 'Variance']])
        return sub[['datadate', 'tic', 'fyearq', 'fqtr', 'conm', 'datadate', 'rdq', 'r2_1', 'r36_13', 'r12_7', 'r12_2', 'ST_Rev', 'LT_Rev', 'AT', 'LME', 'LTurnover', 'Beta', 'MktBeta', 'IdioVol', 'Resid_Var', 'SUV', 'Rel2High', 'ROE', 'Variance']]
    except Exception as e:
        logging.exception(e)
        raise

#df2 = list(map(single_sub, [i for _, i in df.groupby('tic')]))
df2 = process_map(single_sub, [i for _, i in df.groupby('tic')], max_workers=os.cpu_count()-2, chunksize=10)
#df2 = pd.concat(df2,axis=0)
#df2.head()

  0%|          | 0/32354 [00:00<?, ?it/s]

In [ ]:
df2 = pd.concat(df2,axis=0)

In [ ]:
df2.to_csv('features.csv')